<a href="https://colab.research.google.com/github/MatteoMakovec/Recommender_Systems/blob/main/Recommender_system_algorithms.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
## EVALUATIONS
split_percentage = 0.8
num_int = URL_all.nnz

train_mask = np.random.choice([True, False], num_int, p=[split_percentage, 1-split_percentage])
URM_train = sps.csr_matrix(URM_all.data[train_mask], (URM_all.row[train_mask], URM_all.column[train_mask]))

def precision(recommended_items, relevant_items):
  is_relevant = np.ed1d(recommended_items, relevant_items, assume_unique = False)
  precision_score = np.sum(is_relevant, dtype=np.float32) / len(is_relevant)
  return precision_score

def recall(recommended_items, relevant_items):
  is_relevant = np.ed1d(recommended_items, relevant_items, assume_unique = True)
  recall_score = np.sum(is_relevant, dtype=np.float32) / relevant_items.shape[0]
  return recall_score

def MAP(recommended_items, relevant_items):
  is_relevant = np.ed1d(recommended_items, relevant_items, assume_unique = True)
  p_at_k = is_relevant*np.cumsum(is_relevant, dtype=np.float32) / (1+np.arange(is_relevant.shape[0]))
  map_score = np.sum(p_at_k) / np.min([relevant_items.shape[0], is_relevant.shape[0]])
  return map_score

SyntaxError: ignored

In [4]:
## RANDOM RECOMMENDER: TOP-N ITEMS
def fit(self, URM_train):
  item_popularity = np.ediff1d(URM_all.tocsc().indptr)
  self.popular_items = np.argsort(item_popularity)
  self.popular_items = np.flip(self.popular_items, axis = 0)

def recommend(self, user_id, k=5):
  seen_items = self.URM_train.indices[self.URM_train.indptr[user_id]:self.URM_train.indptr[userid+1]]
  unseen_items_mask = np.in1d(self.popular_items, seen_items, assume_unique=True, invert=True)
  unseen_items = self.popular_items[unseen_items_mask]
  recommended_items = self.popular_items[0:k]
  return recommended_items


In [5]:
## GLOBAL EFFECTS
def fit(self, URM_train):
  global_average = np.mean(self.URM_train.data())

  # global average
  URM_train_unbiased = self.URM_train.copy()
  URM_train_unbiased.data -= global_average

  # user bias
  user_mean = URM_train_unbiased.mean(axis=1)
  user_mean = np.array(user_mean).squeeze()

  for user_id in range(len(user_mean)):
    start_position = URM_train_unbiased.indptr[user_id]
    end_position = URM_train_unbiased.indptr[user_id+1]
    URM_train_unbiased.data[start_position:end_position] -= user_mean[user_id]

  # item bias
  item_mean = URM_train_unbiased.mean(axis=0)
  item_mean = np.array(item_mean).squeeze()

  self.best_rated = np.argsort(item_mean)
  self.best_rated = np.flip(self.best_rated, axis = 0)

def recommend(self, user_id, k=10):
  seen_items = self.URM_train.indices[user_id]:self.URM_train.indices[user_id+1]
  unseen_items_mask = np.id1(self.best_rated, seen_items, invert=True)
  unseen_items = self.best_rated[unseen_items_mask]
  recommended_items = unseen_items[0:k]
  return recommended_items

SyntaxError: ignored

In [6]:
## GLOBAL EFFECTS
def fit(self, URM_train):
  global_average = np.mean(self.URM_train.data())

  # global average
  URM_train_unbiased = self.URM_train.copy()
  URM_train_unbiased.data -= global_average

  # user bias
  user_mean = URM_train_unbiased.mean(axis=1)
  user_mean = np.array(user_mean).squeeze()

  for user_id in range(len(user_mean)):
    start_position = URM_train_unbiased.indptr[user_id]
    end_position = URM_train_unbiased.indptr[user_id+1]
    URM_train_unbiased.data[start_position:end_position] -= user_mean[user_id]

  # item bias
  item_mean = URM_train_unbiased.mean(axis=0)
  item_mean = np.array(item_mean).squeeze()

  self.best_rated = np.argsort(item_mean)
  self.best_rated = np.flip(self.best_rated, axis = 0)

def recommend(self, user_id, k=10):
  seen_items = self.URM_train.indices[user_id]:self.URM_train.indices[user_id+1]
  unseen_items_mask = np.id1(self.best_rated, seen_items, invert=True)
  unseen_items = self.best_rated[unseen_items_mask]
  recommended_items = unseen_items[0:k]
  return recommended_items

SyntaxError: ignored

In [7]:
## IDF
num_tot_items = ICM_all.shap[0]
items_per_feature = np.ediff1d(ICM_all.indptr)
IDF = np.array(np.log(num_top_items/items_per_feature))
np.sort(IDF)

NameError: ignored

In [8]:
## ITEM-BASED CF
class Item_CF(object):
  def __init__(self, URM):
    self.URM = URM
  
  def fit(self, topK=50, shrink=100, similarity="cosine", normalized=True):
    similarity_object = Compute_Similarity_Python(self.URM, shrink=shrink, topK=topK, similarity=similarity, normalize=normalize)
    self.W_sparse = similarity_object.compute_similarity()

  def recommend(self, user_id, at:None, filter_seen=True):
    user_profile = URM[user_id]
    scores = user_profile.dot(self.W_sarse).toarray().ravel()
    scores = self.filter_seen(user_id, scores)
    ranking = scores.argsort()[::-1]
    return rankin[:at]

  def filter_seen(self, user_id, scores):
    start_pos = self.URM.indptr[user_id]
    end_pos = self.URM.indptr[user_id+1]
    user_profile = self.URM.indice[start_pos:end_pos]
    scores[user_profile] = -np.inf
    return scores

In [9]:
## USER-BASED CF
class Item_CF(object):
  def __init__(self, URM):
    self.URM = URM
  
  def fit(self, topK=50, shrink=100, similarity="cosine", normalized=True):
    similarity_object = Compute_Similarity_Python(self.URM.T, shrink=shrink, topK=topK, similarity=similarity, normalize=normalize)
    self.W_sparse = similarity_object.compute_similarity()

  def recommend(self, user_id, at:None, filter_seen=True):
    scores = self.W_sparse[user_id, :].dot(self.URM).toarray().ravel()
    scores = self.filter_seen(user_id, scores)
    ranking = scores.argsort()[::-1]
    return rankin[:at]

  def filter_seen(self, user_id, scores):
    start_pos = self.URM.indptr[user_id]
    end_pos = self.URM.indptr[user_id+1]
    user_profile = self.URM.indice[start_pos:end_pos]
    scores[user_profile] = -np.inf
    return scores

In [10]:
## IALS
def update_row(interaction_profile, interaction_confidence, Y, YtY, regularization_diagonal):
  Y_interactions = Y[interaction_profile, :]
  A = Y_interactions.T.dot(((interaction_confidence-1)*Y_interactions.T).T)
  B = YtY + A + regularization_diagonal
  return np.dot(np.linalg.inv(B), Y_interactions.T.dot(interaction_confidence))

C_URM_train_csc = C_URM_train.tocsc()
num_factors = 10

user_factors = np.random.random((n_users, num_factors))
item_factors = np.random.random((n_items, num_factors))

VV = item_factors.T.dot(item_factors)
UU = user_factors.T.dot(user_factors)

for epochs in range (10):
  for user_id in range(C_URM_train.shape[0]):
    start_pos = C_URM_train.indptr[user_id]
    end_pos = C_URM_train.indptr[user_id+1]
    user_profile = C_URM_train.indices[start_pos:end_pos]
    user_confidence = C_URM_train.data[start_pos:end_pos]
    user_factors[user_id, :] = update_row(user_profile, user_confidence, item_factors, VV, regularization_diagonal)
  
  for item_id in range(C_URM_train.shape[1]):
    start_pos = C_URM_train_csc.indptr[item_id]
    end_pos = C_URM_train_csc.indptr[item_id+1]
    item_profile = C_URM_train_csc.indices[start_pos:end_pos]
    item_confidence = C_URM_train_csc.data[start_pos:end_pos]
    user_factors[item_id, :] = update_row(item_profile, item_confidence, user_factors, UU, regularization_diagonal)

predicted_rating = np.dot(user_factors[user_id,:], item_factors[item_id,:])

NameError: ignored

In [11]:
## Pure SVD
U, Sigma, VT = randomized_svd(URM_train, n_components=n_factors)
user_factors = np.dot(U, np.diag(Sigma))
item_factors = VT
predicted_rating = np.dot(user_factors[user_id, :], item_factors[:, item_id])

NameError: ignored

In [12]:
## Item SVD
itemitem_similarity = np.dot(VT.T, VT)
predicted_rating_similarity = URM_train[user_id, :].dot(itemitem_similarity[:, item_id])

NameError: ignored

In [19]:
# Gradient Descendent
x_old = 0
x_new = 2
step = 0.1
precision = 1e-4
f = lambda x : x**3-2*x**2
f_gradient = lambda x : 3*x**2-4*x
x_list, y_list = [x_new], [f(x_new)]

while abs(x_new-x_old)>precision:
  x_old = x_new
  s_k = -f_gradient(x_old)
  x_new = x_old + s_k*step

  x_list.append(x_new)
  y_list.append(f(x_new))

  print("Local minimum occurs at: {:.4f}".format(x_new))
  print("Number of steps: ", len(x_list))

Local minimum occurs at: 1.6000
Number of steps:  2
Local minimum occurs at: 1.4720
Number of steps:  3
Local minimum occurs at: 1.4108
Number of steps:  4
Local minimum occurs at: 1.3780
Number of steps:  5
Local minimum occurs at: 1.3595
Number of steps:  6
Local minimum occurs at: 1.3488
Number of steps:  7
Local minimum occurs at: 1.3426
Number of steps:  8
Local minimum occurs at: 1.3388
Number of steps:  9
Local minimum occurs at: 1.3366
Number of steps:  10
Local minimum occurs at: 1.3353
Number of steps:  11
Local minimum occurs at: 1.3345
Number of steps:  12
Local minimum occurs at: 1.3340
Number of steps:  13
Local minimum occurs at: 1.3338
Number of steps:  14
Local minimum occurs at: 1.3336
Number of steps:  15
Local minimum occurs at: 1.3335
Number of steps:  16
Local minimum occurs at: 1.3334
Number of steps:  17


In [14]:
# SLIM MSE
URM_train_coo = URM_train.tocoo()
item_item_similarity = np.zeros((n_items, n_items), dtype=float)
learning_rate = 1e-6
regularization = 1e-6
loss = 0.0

for samples in range (10000):
  sample_index = np.random.randint(URM_train_coo.nnz)
  user_id = URM_train_coo.row[sample_index]
  item_id = URM_train_coo.column[sample_index]
  true_rating = URM_train_coo.data[sample_index]

  predicted_rating = URM_train[user_id].dot(item_item_similarity[:, item_id])[0]

  prediction_error = true_rating - predicted_rating
  loss += prediction_error**2

  item_in_user_profile = URM_train.indices[URM_train.indptr[user_id]:URM_train.indptr[user_id+1]]
  ratings_in_user_profile = URM_train.data[URM_train.indptr[user_id]:URM_train.indptr[user_id+1]]
  item_item_similarity[item_in_user_profile, item_id] += learning_rate * prediction_error * ratings_in_user_profile

NameError: ignored

In [15]:
# Funk SVD
URM_train_coo = URM_train.tocoo()
num_factors = 10
learning_rate = 1e-4
user_factors = np.random.random((n_users, num_factors))
item_factors = np.random.ranodom((n_items, num_factors))
loss = 00

for samples in range(10000):
  sample_index = np.random.randint(URM_train_coo)
  user_id = URM_train_coo.row[sample_index]
  item_id = URM_train_coo.column[sample_index]
  true_rating = URM_train_coo.data[sample_index]

  predicted_rating = np.dot(user_factors[user_id, :], item_factors[:, item_id])

  prediction_error = true_rating - predicted_rating
  loss += prediction_error**2

  H_i = item_factors[item_id, :]
  W_u = user_factors[user_id, :]

  user_factors_update = prediction_error * H_i - regularization * W_u
  item_factors_update = prediction_error * W_u - regularization * H_i

  user_factors[user_id, :] += learning_rate * user_factors_update
  item_factors[item_id, :] += learning_rate * item_factors_update

NameError: ignored